In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from arch import arch_model

In [ ]:
# Variable definition
data_folder = 'Data/' # Folder containing the dataset
n_days = 1640
look_back = 20
# Rolling window forecasting
window_length = 300

In [ ]:
# Preallocate the array
dataset = np.empty((n_days, 0))

# Create list to save assets names
assets = []
for f in sorted(os.listdir(data_folder)):

    # Save assets names
    assets.append(f.replace('.csv', ''))

    # Load data
    asset = pd.read_csv(data_folder + f, sep=',', usecols=[2, 3], engine='python')
    asset = asset.values[:n_days]
 
    # Ensure all data is float
    asset = asset.astype('float32')
    dataset = np.append(dataset, asset, axis=1)

In [ ]:
mse = []
qlike = []

for i in range(len(assets)):
    # load returns
    ret = dataset[:, i*2]
    
    # Specify the model and estimating parameters
    am = arch_model(ret, vol='Garch', p=look_back, q=look_back)

    forecasts = dict()
    forecasts['day'] = []
    forecasts['fcast'] = []
    
    for j in range(len(ret) - window_length):
        # Estimate the model
        res = am.fit(first_obs=j, last_obs=j+window_length, disp='off')
        
        # Make forecasts
        temp = res.forecast(horizon=1).variance
        fcast = temp.iloc[j + window_length - 1]
        forecasts['day'].append(fcast.name)
        forecasts['fcast'].append(fcast.values)

    y_pred = np.array(forecasts['fcast']).reshape(-1)
    y_true = dataset[window_length:, i*2+1]
    
    # plt.figure()
    # plt.plot(forecasts['day'], y_pred, label='forecast')
    # plt.plot(forecasts['day'], y_true, label='expected')
    # plt.ylim([0, 60])
    # plt.legend()
    # plt.show()

    # Calculate the evaluation metrics
    mse_i = (y_true - y_pred) ** 2
    qlike_i = np.log(y_pred) + (y_true / y_pred)
    
    # Save results
    results = np.array([mse_i, qlike_i]).transpose()
    np.savetxt('7-GARCH/' + assets[i] + '.csv', results, delimiter=',',
               header='MSE, Q-LIKE', fmt='%10.5f', comments='')
    
    mse.append(np.mean(mse_i, axis=0))
    qlike.append(np.mean(qlike_i, axis=0))

In [ ]:
# Save results
results = np.array([mse, qlike]).transpose()
np.savetxt('results/7.csv', results, delimiter=',', header='MSE, Q-LIKE', fmt='%10.5f', comments='')

df = pd.DataFrame({'MSE': mse, 'QLIKE': qlike})
print(df.describe())